# Criação, treino e teste do Modelo

## Instalando bibliotecas 

In [1]:
!pip install tensorflow


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


## Leitura e Verificação dos Dados

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

2024-09-11 17:30:33.894411: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-11 17:30:33.902252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 17:30:33.911516: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 17:30:33.914245: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-11 17:30:33.921400: I tensorflow/core/platform/cpu_feature_guar

In [3]:
comum = pd.read_parquet('DF_KNRS_COMUM_PROCESSADO.parquet')
tempo = pd.read_parquet('tempo_de_cada_knr_por_halle.parquet')

df = comum.merge(tempo, on='KNR', how='right')
df = df.dropna()
print(df['TEM_FALHA_ROD'].value_counts())  # Verificando a distribuição da variável alvo
print(df.isnull().sum())
print('\n----------\n')

df = df.dropna()  # Removendo valores nulos
df = df.drop(columns=["QTD_HALLE_ROD", "QTD_HALLE_AGUA", "QTD_HALLE_ZP8", "QTD_HALLE_ZP8R"])

print('\n----------\n')
print(df['TEM_FALHA_ROD'].value_counts())  # Verificando a distribuição da variável alvo
print(df.isnull().sum())

TEM_FALHA_ROD
0.0    12967
1.0      616
Name: count, dtype: int64
KNR                       0
QTD_STATUS_1_OK           0
QTD_STATUS_1_NOK          0
QTD_STATUS_2_OK           0
QTD_STATUS_2_NOK          0
QTD_STATUS_718_OK         0
QTD_STATUS_718_NOK        0
TEMPO_MEDIO               0
MOTOR                     0
COR                       0
QTD_HALLE_                0
QTD_HALLE_AGUA            0
QTD_HALLE_BUY             0
QTD_HALLE_CAB             0
QTD_HALLE_DKA             0
QTD_HALLE_ESPC            0
QTD_HALLE_PROC            0
QTD_HALLE_PROF            0
QTD_HALLE_PVC             0
QTD_HALLE_ROD             0
QTD_HALLE_RUID            0
QTD_HALLE_TLUI            0
QTD_HALLE_ZP5             0
QTD_HALLE_ZP5A            0
QTD_HALLE_ZP6             0
QTD_HALLE_ZP61            0
QTD_HALLE_ZP62            0
QTD_HALLE_ZP7             0
QTD_HALLE_ZP8             0
QTD_HALLE_ZP82            0
QTD_HALLE_ZP8R            0
QTD_SGROUP_#MULTIVALUE    0
QTD_SGROUP_-2             0
QTD_SGROUP

In [4]:
qtd_ocorrencias = df[df['QTD_HALLE_ESPC'] > 1].shape[0]
print(f"Ocorrências de valor 1 na coluna QTD_HALLE_ESPC: {qtd_ocorrencias}")


Ocorrências de valor 1 na coluna QTD_HALLE_ESPC: 248


In [5]:
df.head()

,KNR,QTD_STATUS_1_OK,QTD_STATUS_1_NOK,QTD_STATUS_2_OK,QTD_STATUS_2_NOK,QTD_STATUS_718_OK,QTD_STATUS_718_NOK,TEMPO_MEDIO,MOTOR,COR,...,QTD_SGROUP_2,QTD_SGROUP_4,QTD_SGROUP_5,QTD_SGROUP_9830946,TEM_FALHA_ROD,ZP5_MIN,ZP5A_MIN,ZP61_MIN,ZP6 / ZP62_MIN,CAB_MIN
6890,2023-3016123,7.0,0.0,840.0,0.0,50.0,1.0,50.883333,CWL,2R2R,...,3.0,1.0,0.0,0.0,1.0,1740.483333,5605.383333,102.200000,25.883333,57.666667
6891,2023-3016123,7.0,0.0,803.0,0.0,48.0,1.0,92.250000,CWL,2R2R,...,3.0,1.0,0.0,0.0,1.0,1740.483333,5605.383333,102.200000,25.883333,57.666667
7010,2023-3016194,7.0,0.0,846.0,9.0,58.0,2.0,52.683333,DHS,0Q0Q,...,7.0,2.0,0.0,0.0,0.0,2582.033333,5817.316667,53.133333,23.350000,11454.383333
7011,2023-3016194,7.0,0.0,802.0,9.0,56.0,2.0,73.350000,DHS,0Q0Q,...,7.0,2.0,0.0,0.0,0.0,2582.033333,5817.316667,53.133333,23.350000,11454.383333
7252,2023-3026027,7.0,0.0,850.0,6.0,53.0,4.0,48.633333,DHS,2R2R,...,7.0,3.0,0.0,0.0,0.0,1772.450000,5611.933333,36.833333,25.966667,4538.233333


## Pré-processamento dos dados

In [6]:
from sklearn.preprocessing import MinMaxScaler
# Criando variáveis dummies
df = pd.get_dummies(df, columns=['COR', 'MOTOR'], drop_first=True)

# Removendo colunas desnecessárias
df = df.drop(columns=["KNR"])


# Exemplo de DataFrame com colunas numéricas e de string
# Suponha que df seja o seu DataFrame original

# Separar as colunas numéricas
df_numerical = df.select_dtypes(include=['float64', 'int64'])

# Separar as colunas não numéricas
df_non_numerical = df.select_dtypes(exclude=['float64', 'int64'])

# Aplicar o MinMaxScaler nas colunas numéricas
scaler = MinMaxScaler()
df_numerical_scaled = pd.DataFrame(scaler.fit_transform(df_numerical), columns=df_numerical.columns)



In [7]:
df_numerical_scaled.head()

,QTD_STATUS_1_OK,QTD_STATUS_1_NOK,QTD_STATUS_2_OK,QTD_STATUS_2_NOK,QTD_STATUS_718_OK,QTD_STATUS_718_NOK,TEMPO_MEDIO,QTD_HALLE_,QTD_HALLE_BUY,QTD_HALLE_CAB,...,QTD_SGROUP_2,QTD_SGROUP_4,QTD_SGROUP_5,QTD_SGROUP_9830946,TEM_FALHA_ROD,ZP5_MIN,ZP5A_MIN,ZP61_MIN,ZP6 / ZP62_MIN,CAB_MIN
0,0.466667,0.0,0.752688,0.000000,0.263158,0.014085,0.002383,0.0,0.0,0.0,...,0.157895,0.055556,0.0,0.0,1.0,0.022740,0.103751,0.005534,0.000860,0.002519
1,0.466667,0.0,0.719534,0.000000,0.252632,0.014085,0.004320,0.0,0.0,0.0,...,0.157895,0.055556,0.0,0.0,1.0,0.022740,0.103751,0.005534,0.000860,0.002519
2,0.466667,0.0,0.758065,0.089109,0.305263,0.028169,0.002467,0.0,0.0,0.0,...,0.368421,0.111111,0.0,0.0,0.0,0.041130,0.107964,0.002762,0.000749,0.520787
3,0.466667,0.0,0.718638,0.089109,0.294737,0.028169,0.003435,0.0,0.0,0.0,...,0.368421,0.111111,0.0,0.0,0.0,0.041130,0.107964,0.002762,0.000749,0.520787
4,0.466667,0.0,0.761649,0.059406,0.278947,0.056338,0.002277,0.0,0.0,0.0,...,0.368421,0.166667,0.0,0.0,0.0,0.023439,0.103882,0.001841,0.000864,0.206273


## Balanceamento das Classes

In [8]:
from imblearn.over_sampling import SMOTE


from sklearn.model_selection import train_test_split
X = df_numerical_scaled.drop(columns=['TEM_FALHA_ROD'])  # Removendo a variável alvo, mantendo apenas as variáveis preditoras
y = df_numerical_scaled['TEM_FALHA_ROD']  # Definindo a variável alvo

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)# Initialize SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)# Apply SMOTE to the training data
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

## Dividindo o dataset entre treino e teste

In [9]:
from sklearn.model_selection import train_test_split  # Importa a função para dividir os dados em conjuntos de treino e teste

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)  
# Divide os dados balanceados em conjuntos de treino (80%) e teste (20%), com uma semente de aleatoriedade fixa para reprodução

## Redimensionamento dos dados

In [10]:
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))  
# Altera a forma de X_train para (n amostras, 1, n características) para compatibilidade com redes neurais que esperam uma dimensão adicional

X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))  
# Altera a forma de X_test de maneira semelhante

X_train = np.array(X_train, dtype=np.float32)  
# Converte X_train para um array NumPy com tipo de dado float32

y_train = np.array(y_train, dtype=np.float32)  
# Converte y_train para um array NumPy com tipo de dado float32

## Construção dos Modelos (LSTM e GRU)

### Modelo GRU

In [11]:
model_gru = Sequential()  # Cria um modelo sequencial, que é uma pilha linear de camadas

model_gru.add(GRU(50, input_shape=(X_train.shape[1], X_train.shape[2])))  
# Adiciona uma camada GRU com 50 unidades. A forma de entrada é (n timesteps, n características)

model_gru.add(Dropout(0.2))  
# Adiciona uma camada Dropout para prevenir overfitting, desativando aleatoriamente 20% dos neurônios durante o treinamento

model_gru.add(Dense(1, activation='sigmoid'))  
# Adiciona uma camada densa de saída com 1 neurônio e ativação sigmoide para uma tarefa de classificação binária

model_gru.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])  
# Compila o modelo usando o otimizador Adam com taxa de aprendizado de 0.001, a função de perda de entropia cruzada binária e a métrica de acurácia

I0000 00:00:1726086636.440172   85277 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-11 17:30:36.456954: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/crz/Github/2024-2A-T08-EC07-G05/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init_

In [12]:
# Treinando o modelo GRU
history_gru = model_gru.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.2)

Epoch 1/500
364/364 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5425 - loss: 0.6886 - val_accuracy: 0.6334 - val_loss: 0.6668
Epoch 2/500
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - accuracy: 0.6294 - loss: 0.6632 - val_accuracy: 0.6778 - val_loss: 0.6265
Epoch 3/500
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - accuracy: 0.6756 - loss: 0.6288 - val_accuracy: 0.7246 - val_loss: 0.5896
Epoch 4/500
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 713us/step - accuracy: 0.7142 - loss: 0.5989 - val_accuracy: 0.7631 - val_loss: 0.5555
Epoch 5/500
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 932us/step - accuracy: 0.7522 - loss: 0.5602 - val_accuracy: 0.8367 - val_loss: 0.4997
Epoch 6/500
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step - accuracy: 0.8261 - loss: 0.5053 - val_accuracy: 0.8910 - val_loss: 0.4425
Epoch 7/500
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 865us/step - accuracy: 0.8813 - loss: 0.4442 - val_accuracy: 0.9285 - val_loss: 0.3845
Epoch 8/500
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - accuracy: 0.9162 - loss: 0.3

### Modelo LSTM

In [13]:
# Definir o modelo LSTM
model_lstm = Sequential()  # Cria um modelo sequencial para o LSTM

# Camada LSTM
model_lstm.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))  
# Adiciona uma camada LSTM com 50 unidades. A forma de entrada é (n timesteps, n características)

model_lstm.add(Dropout(0.2))  
# Adiciona uma camada Dropout para prevenir overfitting, desativando aleatoriamente 20% dos neurônios durante o treinamento

# Camada de saída
model_lstm.add(Dense(1, activation='sigmoid'))  
# Adiciona uma camada densa de saída com 1 neurônio e ativação sigmoide para uma tarefa de classificação binária

# Compilar o modelo
model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])  
# Compila o modelo usando o otimizador Adam com taxa de aprendizado de 0.001, a função de perda de entropia cruzada binária e a métrica de acurácia

# Resumo do modelo
model_lstm.summary()  # Exibe um resumo da arquitetura do modelo

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        18,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,051 (70.51 KB)

 Trainable params: 18,051 (70.51 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Treinando o modelo LSTM
history_lstm = model_lstm.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2)

Epoch 1/200
364/364 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5547 - loss: 0.6899 - val_accuracy: 0.6510 - val_loss: 0.6707
Epoch 2/200
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - accuracy: 0.6322 - loss: 0.6657 - val_accuracy: 0.6802 - val_loss: 0.6243
Epoch 3/200
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - accuracy: 0.6767 - loss: 0.6251 - val_accuracy: 0.7084 - val_loss: 0.5899
Epoch 4/200
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step - accuracy: 0.7076 - loss: 0.5952 - val_accuracy: 0.7703 - val_loss: 0.5462
Epoch 5/200
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step - accuracy: 0.7751 - loss: 0.5481 - val_accuracy: 0.8380 - val_loss: 0.4936
Epoch 6/200
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 688us/step - accuracy: 0.8265 - loss: 0.4988 - val_accuracy: 0.8948 - val_loss: 0.4329
Epoch 7/200
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step - accuracy: 0.8776 - loss: 0.4405 - val_accuracy: 0.9264 - val_loss: 0.3723
Epoch 8/200
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step - accuracy: 0.9157 - loss: 0.3

## Avaliando o Modelo

### GRU

In [15]:
# Converter X_test e y_test para float32
X_test = np.array(X_test).astype('float32')
y_test = np.array(y_test).astype('float32')

# Avaliar o modelo GRU com os dados de teste
loss_gru, accuracy_gru = model_gru.evaluate(X_test, y_test)
print(f'Test Loss GRU: {loss_gru}')
print(f'Test Accuracy GRU: {accuracy_gru}')

# Fazer previsões com o modelo GRU
y_pred_gru = model_gru.predict(X_test)
y_pred_gru = (y_pred_gru > 0.5).astype(int)

# Exibir o relatório de classificação
print(classification_report(y_test, y_pred_gru))


114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 306us/step - accuracy: 0.9820 - loss: 0.0969
Test Loss GRU: 0.0934840515255928
Test Accuracy GRU: 0.9810230731964111
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
              precision    recall  f1-score   support

         0.0       1.00      0.96      0.98      1823
         1.0       0.97      1.00      0.98      1813

    accuracy                           0.98      3636
   macro avg       0.98      0.98      0.98      3636
weighted avg       0.98      0.98      0.98      3636



### LSTM

In [16]:
# Avaliar o modelo LSTM com os dados de teste
loss_lstm, accuracy_lstm = model_lstm.evaluate(X_test, y_test)
print(f'Test Loss LSTM: {loss_lstm}')
print(f'Test Accuracy LSTM: {accuracy_lstm}')

# Fazer previsões com o modelo LSTM
y_pred_lstm = model_lstm.predict(X_test)
y_pred_lstm = (y_pred_lstm > 0.5).astype(int)

# Exibir o relatório de classificação
print(classification_report(y_test, y_pred_lstm))

114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 344us/step - accuracy: 0.9740 - loss: 0.1446
Test Loss LSTM: 0.1315179020166397
Test Accuracy LSTM: 0.9757975935935974
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step
              precision    recall  f1-score   support

         0.0       1.00      0.95      0.98      1823
         1.0       0.95      1.00      0.98      1813

    accuracy                           0.98      3636
   macro avg       0.98      0.98      0.98      3636
weighted avg       0.98      0.98      0.98      3636

